In [32]:
import time
import seaborn as sns
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from collections import Counter
from scipy.spatial import cKDTree as KDTree
import os
import random
import numpy as np
import pandas as pd 
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, fcluster
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import OPTICS
from glob import glob 
from tqdm import tqdm
from scipy.spatial.distance import pdist , squareform
np.random.seed(5)
import itertools

In [53]:
def generate_data(size_list, proba_tupple):
    size_list = size_list
    probas = proba_tupple
    number_of_iteration = tuple([int(size_list*x) for x in probas])

    list_of_iteration = []
    for i in range(len(number_of_iteration)):
        a = [i]*number_of_iteration[i]
        list_of_iteration.append(a)
    flat_list = [item for sublist in list_of_iteration for item in sublist]

    return flat_list

def compute_binary_matrix(mat):
    nx, ny = mat.shape
    res = 0 * mat[:, 0]
    for ix in range(ny):
        res += mat[: , ix] * 2 ** ix
    return res  

def reverse_binary_matrix(list_of_iteration, dataframe_to_match,column):
    list_binary = []
    for value in flat_list:
        match = dataframe_to_match.loc[dataframe_to_match['%s'%column].isin([value]),[0,1]].values.tolist()
        list_binary.append(match)
    list_binary = [item for sublist in list_binary for item in sublist]

    return list_binary


In [59]:
size_list = 100
proba_tupple = (0.85, 0.05,0.05,0.05)

data_generated = generate_data(size_list,proba_tupple)

equivalent = np.array(list(itertools.product([0, 1], repeat=2)))
conversion = compute_binary_matrix(equivalent)


binary_result = pd.DataFrame(equivalent)
binary_result['Match_number'] = pd.Series(conversion)
binary_result.sort_values(by =['Match_number'], inplace = True)

frame_binary = pd.DataFrame(reverse_binary_matrix(data_generated, dataframe_to_match = binary_result, column = 'Match_number'))

In [60]:
frame_binary

,0,1
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
95,1,1
96,1,1
97,1,1
98,1,1


In [31]:
pd.DataFrame(list_binary)

,0,1
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
95,1,1
96,1,1
97,1,1
98,1,1


In [2]:

#Generate patients
number_of_patients = 50
number_of_cell_phenotypes = 7
number_of_feature_to_generate = 1
#Patient_phenotype
patient_phenotypes = 2
total_number_of_cell = 10000

n_sample = 5000 #Number of cell subset randomly chosen
strating_point = 0
number_of_cluster = 2 #Number_of_cluster_for_k_means
max_val = 0.01
#std for phenotype 1 and 2 updated at every run
cluster_std_1 = 0.1
cluster_std_2= 0.2

        #Define parameters for LDA
diviser_of_matrix = 100
number_of_calculation = 50
number_of_topic_to_test = 10
        #Parameters for cutting dendrogram
cutting_tree = 1

In [4]:

left_part, right_part = 1000,9000
cell_distrib = [left_part,right_part]
cell_distrib_2 = [right_part,left_part]

left_part_2, right_part_2 = 1000,9000
cell_2_distrib = [left_part_2, right_part_2]
cell_2_distrib_2 = [right_part_2, left_part_2]

phenotype_code = []

for patient in tqdm(range(number_of_patients)):
    random_state = 41
    phenotype_continuous_data = []
    phenotype_binary_data = []
    if patient < number_of_patients/patient_phenotypes:
        phenotype_code.append(1)
        X, y = make_blobs(n_samples=np.array(cell_distrib), centers=None, n_features= number_of_feature_to_generate, cluster_std=cluster_std_1, random_state=random_state)
        phenotype_continuous_data.append(X)
        phenotype_binary_data.append(y)
        X, y = make_blobs(n_samples=np.array(cell_2_distrib), centers=None, n_features= number_of_feature_to_generate, cluster_std=cluster_std_1, random_state=random_state)
        phenotype_continuous_data.append(X)
        phenotype_binary_data.append(y)
        random_state =+ 10
        cluster_std_1 =+ random.uniform(0, 0.5)
    else:
        phenotype_code.append(2)
        X, y = make_blobs(n_samples=np.array(cell_distrib_2), centers=None, n_features= number_of_feature_to_generate, cluster_std=cluster_std_2, random_state=random_state)
        phenotype_continuous_data.append(X)
        phenotype_binary_data.append(y)
        X, y = make_blobs(n_samples=np.array(cell_2_distrib_2), centers=None, n_features= number_of_feature_to_generate, cluster_std=cluster_std_1, random_state=random_state)
        phenotype_continuous_data.append(X)
        phenotype_binary_data.append(y)
        random_state =+ 10
        cluster_std_1 =+ random.uniform(0, 0.5)


    phenotype_continuous_data_patient = pd.DataFrame(list(map(np.ravel,phenotype_continuous_data))).T
    phenotype_binary_data_patient = pd.DataFrame(list(map(np.ravel,phenotype_binary_data))).T

 32%|███▏      | 16/50 [00:09<00:20,  1.70it/s]


KeyboardInterrupt: 

In [71]:
first_part, second_part = 6000,4000
first_cell_distrib = np.round([first_part,second_part])

percentage_1 = 0.4
second_part_column_wth_1, second_part_column_wth_0 = int(percentage_1 * first_part), int((1-percentage_1)*first_part)
second_part_1_cell_distrib = [second_part_column_wth_1, second_part_column_wth_0]

percentage_2 = 0.2
third_part_column_with_1, third_part_column_wth_0 = int(percentage_2 * second_part), int((1-percentage_2) * second_part)
second_part_2_cell_distrib = [third_part_column_with_1, third_part_column_wth_0]

In [77]:
print(second_part_1_cell_distrib,second_part_2_cell_distrib)

[2400, 3600] [800, 3200]


In [75]:
phenotype_technique_x = []
phenotype_technique_y = []

#Append First row data
X, y = make_blobs(n_samples=first_cell_distrib, centers=None, n_features= number_of_feature_to_generate, cluster_std=cluster_std_1, random_state=random_state)
phenotype_technique_x.append(X)
phenotype_technique_y.append(y)

#second column generation
X_1, y_1 = make_blobs(n_samples=second_part_1_cell_distrib, centers=None, n_features= number_of_feature_to_generate, cluster_std=cluster_std_1, random_state=random_state)
phenotype_continuous_data.append(X)
phenotype_binary_data.append(y)
X_2,y_2 = make_blobs(n_samples=second_part_2_cell_distrib, centers=None, n_features= number_of_feature_to_generate, cluster_std=cluster_std_1, random_state=random_state)

In [17]:
first_part, second_part = 100,300
first_cell_distrib = np.round([first_part,second_part])

percentage_1 = 0.4
second_part_column_wth_1, second_part_column_wth_0 = int(percentage_1 * first_part), int((1-percentage_1)*first_part)
second_part_1_cell_distrib = [second_part_column_wth_1, second_part_column_wth_0]

percentage_2 = 0.2
third_part_column_with_1, third_part_column_wth_0 = int(percentage_2 * second_part), int((1-percentage_2) * second_part)
second_part_2_cell_distrib = [third_part_column_with_1, third_part_column_wth_0]

In [18]:
first_colum = [1] * first_part + [0] *second_part
second_column_test = [1] * second_part_1_cell_distrib[0]+ [0] * second_part_1_cell_distrib[1] + [1] * second_part_2_cell_distrib[0] + [0] *second_part_2_cell_distrib[1]

In [19]:
test = pd.DataFrame( {'Col1': first_colum, 'Col2': second_column})

In [164]:
buckets[1]

1

In [157]:
frame = { 'Author': pd.Series(buckets), 'Article': pd.Series(second_column) }
result = pd.DataFrame(frame)


TypeError: 'tuple' object is not callable

In [158]:
tuple

tuple

In [4]:
left_part, right_part = 100,300
first_cell_distrib = np.round([left_part,right_part])

percentage_1 = 0.4
right_part_column_wth_1, right_part_column_wth_0 = int(percentage_1 * left_part), int((1-percentage_1)*left_part)
right_part_1_cell_distrib = [right_part_column_wth_1, right_part_column_wth_0]

percentage_2 = 0.2
third_part_column_with_1, third_part_column_wth_0 = int(percentage_2 * right_part), int((1-percentage_2) * right_part)
right_part_2_cell_distrib = [third_part_column_with_1, third_part_column_wth_0]

In [5]:
first_colum = [1] * first_part + [0] *second_part
second_column_test = [1] * second_part_1_cell_distrib[0]+ [0] * second_part_1_cell_distrib[1] + [1] * second_part_2_cell_distrib[0] + [0] *second_part_2_cell_distrib[1]

NameError: name 'first_part' is not defined

In [5]:

left_part, right_part = 100,300
first_cell_distrib = np.round([left_part,right_part])

percentage_1 = 0.4
right_part_column_wth_1, right_part_column_wth_0 = int(percentage_1 * left_part), int((1-percentage_1)*left_part)
right_part_1_cell_distrib = [right_part_column_wth_1, right_part_column_wth_0]

percentage_2 = 0.2
third_part_column_with_1, third_part_column_wth_0 = int(percentage_2 * right_part), int((1-percentage_2) * right_part)
right_part_2_cell_distrib = [third_part_column_with_1, third_part_column_wth_0]

first_colum = [1] * left_part + [0] *right_part
second_column_test = [1] * right_part_1_cell_distrib[0]+ [0] * right_part_1_cell_distrib[1] + [1] * right_part_2_cell_distrib[0] + [0] *right_part_2_cell_distrib[1]

In [7]:
test = pd.DataFrame( {'Col1': first_colum, 'Col2': second_column_test})

In [8]:
test

,Col1,Col2
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
395,0,0
396,0,0
397,0,0
398,0,0
